In [ ]:
!pip install gradio

In [ ]:
import gradio
import gradio as gr

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.xception import preprocess_input as xcp_preprocess_input

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
# import torchvision.models as models
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

from PIL import Image
from torchvision import transforms

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=1):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b3(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        #self.model = timm.create_model('vgg16', pretrained=True, num_classes=1)

    def forward(self, x):
        x = self.backbone(x)
        x = F.sigmoid(self.classifier(x))
        return x
        #x = F.sigmoid(self.model(x))
        #return x

In [ ]:
# 토치 모델 불러오기 
infer_model = BaseModel()
model = infer_model.load_state_dict(torch.load('/content/drive/MyDrive/findco/비타민 자율프로젝트(1조)/cv_models/review_model.pt'))
infer_model.to(device)

In [ ]:
# torch
def predict(inp):
  inp = transforms.ToTensor()(inp).unsqueeze(0)
  with torch.no_grad():
    prediction = torch.nn.functional.sigmoid(model(inp)[0], dim=0) #softmax
  preds = (prediction[:,0] >= 0.7).astype(int)
  if preds == 0:
    return "(%.2f) 광고가 아닙니다." % (1-prediction)
  else:
    return "(%.2f) 광고입니다." % (prediction)

In [ ]:
# torch
gr.Interface(fn=predict, 
             inputs=gr.Image(type="pil"),
             outputs=gr.Label(num_top_classes=2)).launch(debug='True',share=True)

In [ ]:
# 텐서 모델 불러오기
from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/findco/best_model.h5')

In [ ]:
# tensor
def classify_image(img):
  img = img.reshape((-1, 400, 400, 3))
  img = xcp_preprocess_input(img)
  prediction = model.predict(img)
  prediction = prediction[0]
  #preds = (prediction[:,0] >= 0.9).astype(int)
  answer = lambda x: 1 if x>= 0.8 else 0
  preds = answer(prediction)
  if preds == 0:
    return "(%.2f) 광고가 아닙니다." % (1-prediction)
  elif preds == 1:
    return "(%.2f) 광고입니다." % (prediction)

In [ ]:
#im = gradio.inputs.Image(shape=(400, 400), image_mode='L', invert_colors=False, source="upload")
# tensor
iface = gr.Interface(
    fn = classify_image, 
    inputs =gr.inputs.Image(shape=(400, 400)), 
    outputs=gr.outputs.Label(num_top_classes=2)
)
iface.launch(debug='True',share=True)

In [ ]:
#Ensemble
model_nlp = tf.keras.models.load_model(
    './4d/mixup/0020-0.0535-0.9920.hdf5'
)

model2_cv = tf.keras.models.load_model(
    './4d/cutmix/0018-0.1665-0.9941.hdf5'
)

preds1 = model_nlp.predict(testloader)
preds2 = model_cv.predict(testloader)
preds = tf.concat([preds1[tf.newaxis, :, :], preds2[tf.newaxis, :, :]], axis=0)
preds = tf.reduce_mean(preds, axis=0)